In [7]:
!pip install polars plotly great_tables nbformat

In [5]:
import re
import polars as pl
import plotly.graph_objs as go

In [6]:
logfile = "PegaRULES.log"
rows = []
# Regex:
# - Time: Success (xxx us|x ms)
# - After values: the query starts
pattern = re.compile(
    r'Success \((\d+)\s*(us|ms)\).*?\]\s+(.*)$',
    re.MULTILINE
)

def parse_time(value, unit):
    value = int(value)
    if unit == "us":
        return value / 1000
    return value

In [7]:
with open(logfile, "r") as f:
    for line in f:
        m = pattern.search(line)
        if m:
            time_ms = parse_time(m.group(1), m.group(2))
            query = m.group(3).strip()
            query_norm = re.sub(r'\s+', ' ', query)
            rows.append({"query": query_norm, "exec_time_ms": time_ms})

In [8]:


df = pl.DataFrame(rows)
percentiles = [0.5, 0.75, 0.9, 0.95, 0.99]
exprs = [
    pl.col("exec_time_ms").mean().alias("avg"),
] + [
    pl.col("exec_time_ms").quantile(p, "nearest").alias(f"p{int(p*100)}")
    for p in percentiles
]

stats = (
    df.group_by("query")
    .agg(exprs)
    .sort("avg", descending=True)
)

top_n = 1000
stats_top = stats.head(top_n)

fig = go.Figure()
for p in ["avg", "p50", "p75", "p90", "p95", "p99"]:
    fig.add_trace(go.Bar(
        x=stats_top.select(pl.col('query').str.slice(0, length=50))["query"].to_list(),
        y=stats_top[p].to_list(),
        name=p,
    ))

fig.update_layout(
    title="Query Execution Times: avg, 50/75/90/95/99 percentiles (Top Slowest Queries)",
    xaxis_title="Query",
    yaxis_title="Execution Time (ms)",
    barmode="group",
    legend_title="Metric",
    height=800,
    width=1200,
)

fig.show()


In [9]:
from great_tables import loc, style
import polars.selectors as cs
stats_top.style.tab_style(
    style.fill("yellow"),
    loc.body(rows=pl.col("avg") == pl.col("avg").max()),
).tab_spanner(
    "Query", cs.starts_with("query")
).tab_spanner(
    "Precentiles", cs.starts_with("p")
).fmt_number(cs.starts_with("avg"), decimals=2)    

GT(_tbl_data=shape: (88, 7)
┌─────────────────────────────────┬──────────┬───────┬───────┬───────┬───────┬───────┐
│ query                           ┆ avg      ┆ p50   ┆ p75   ┆ p90   ┆ p95   ┆ p99   │
│ ---                             ┆ ---      ┆ ---   ┆ ---   ┆ ---   ┆ ---   ┆ ---   │
│ str                             ┆ f64      ┆ f64   ┆ f64   ┆ f64   ┆ f64   ┆ f64   │
╞═════════════════════════════════╪══════════╪═══════╪═══════╪═══════╪═══════╪═══════╡
│ SELECT p,nid,mid,d,writetime(d… ┆ 34.0     ┆ 34.0  ┆ 34.0  ┆ 34.0  ┆ 34.0  ┆ 34.0  │
│ select data_center from system… ┆ 30.0     ┆ 30.0  ┆ 30.0  ┆ 30.0  ┆ 30.0  ┆ 30.0  │
│ INSERT INTO adm.adm_scoringmod… ┆ 27.0     ┆ 27.0  ┆ 27.0  ┆ 27.0  ┆ 27.0  ┆ 27.0  │
│ UPDATE adm.adm_factory_handled… ┆ 26.0     ┆ 26.0  ┆ 26.0  ┆ 26.0  ┆ 26.0  ┆ 26.0  │
│ INSERT INTO adm_commitlog.adm_… ┆ 14.0     ┆ 14.0  ┆ 14.0  ┆ 14.0  ┆ 14.0  ┆ 14.0  │
│ …                               ┆ …        ┆ …     ┆ …     ┆ …     ┆ …     ┆ …     │
│ SELECT pysubjectid,pysubjectty… ┆ 0.41854  ┆ 0.375 ┆ 0.447 ┆ 0.606 ┆ 0.666 ┆ 0.824 │
│ SELECT ky,k2,vl FROM aggregati… ┆ 0.411242 ┆ 0.365 ┆ 0.477 ┆ 0.606 ┆ 0.648 ┆ 0.794 │
│ SELECT vl FROM states."stores_… ┆ 0.405832 ┆ 0.367 ┆ 0.434 ┆ 0.584 ┆ 0.688 ┆ 1.0   │
│ SELECT k2,vl FROM states."stor… ┆ 0.40545  ┆ 0.364 ┆ 0.416 ┆ 0.593 ┆ 0.69  ┆ 1.0   │
│ SELECT pysubjectid,pysubjectty… ┆ 0.380057 ┆ 0.359 ┆ 0.423 ┆ 0.507 ┆ 0.606 ┆ 0.802 │
└─────────────────────────────────┴──────────┴───────┴───────┴───────┴───────┴───────┘, _body=<great_tables._gt_data.Body object at 0x158688d70>, _boxhead=Boxhead([ColInfo(var='query', type=<ColInfoTypeEnum.default: 1>, column_label='query', column_align='left', column_width=None), ColInfo(var='avg', type=<ColInfoTypeEnum.default: 1>, column_label='avg', column_align='right', column_width=None), ColInfo(var='p50', type=<ColInfoTypeEnum.default: 1>, column_label='p50', column_align='right', column_width=None), ColInfo(var='p75', type=<ColInfoTypeEnum.default: 1>, column_label='p75', column_align='right', column_width=None), ColInfo(var='p90', type=<ColInfoTypeEnum.default: 1>, column_label='p90', column_align='right', column_width=None), ColInfo(var='p95', type=<ColInfoTypeEnum.default: 1>, column_label='p95', column_align='right', column_width=None), ColInfo(var='p99', type=<ColInfoTypeEnum.default: 1>, column_label='p99', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x158688ad0>, _spanners=Spanners([SpannerInfo(spanner_id='Query', spanner_level=0, spanner_label='Query', spanner_units=None, spanner_pattern=None, vars=['query'], built=None), SpannerInfo(spanner_id='Precentiles', spanner_level=0, spanner_label='Precentiles', spanner_units=None, spanner_pattern=None, vars=['p50', 'p75', 'p90', 'p95', 'p99'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns=None, rows=<Expr ['[(col("avg")) == (col("avg").m…'] at 0x158665450>, mask=None), grpname=None, colname='query', rownum=0, colnum=None, styles=[CellStyleFill(color='yellow')]), StyleInfo(locname=LocBody(columns=None, rows=<Expr ['[(col("avg")) == (col("avg").m…'] at 0x158665450>, mask=None), grpname=None, colname='avg', rownum=0, colnum=None, styles=[CellStyleFill(color='yellow')]), StyleInfo(locname=LocBody(columns=None, rows=<Expr ['[(col("avg")) == (col("avg").m…'] at 0x158665450>, mask=None), grpname=None, colname='p50', rownum=0, colnum=None, styles=[CellStyleFill(color='yellow')]), StyleInfo(locname=LocBody(columns=None, rows=<Expr ['[(col("avg")) == (col("avg").m…'] at 0x158665450>, mask=None), grpname=None, colname='p75', rownum=0, colnum=None, styles=[CellStyleFill(color='yellow')]), StyleInfo(locname=LocBody(columns=None, rows=<Expr ['[(col("avg")) == (col("avg").m…'] at 0x158665450>, mask=None), grpname=None, colname='p90', rownum=0, colnum=None, styles=[CellStyleFill(color='yellow')]), StyleInfo(locname=LocBody(col